In [1]:
import numpy as np
import itertools

In [2]:
def makeSymbolList(numSymbolOnReel, numSymbolTotal, precision):
    fenceposts = [round(i, int(np.log10(precision))) for i in list(np.random.random(size=numSymbolOnReel - 1))]
    fenceposts = sorted([0] + fenceposts + [1])
    symbols = np.random.choice(numSymbolTotal, size=numSymbolOnReel, replace=False)
    symbolList = [(symbols[i],round((fenceposts[i+1] - fenceposts[i]), 
                                    int(np.log10(precision)))) for i in range(numSymbolOnReel)]
    return symbolList
    
def makeMachine(numSymbolList, numSymbolsTotal, precision):
    if numSymbolsTotal < max(numSymbolList):
        print("Total number of symbols too small...")
        return
    numReel = len(numSymbolList)
    machineDefList = [sorted(makeSymbolList(numSymbolList[i], 
                                            numSymbolsTotal, precision),
                             key=lambda x: x[1], reverse = True) for i in range(numReel)]
    print("Probabilities sum to 1? " + str([sum([j[1] for j in i]) for i in machineDefList]))
    return machineDefList

def evaluateMachine(machineDefList):
    numReel = len(machineDefList)
    probDict = {i:0 for i in range(1, numReel + 1)}
    for thisTuple in itertools.product(*machineDefList):
        thisCount = numReel + 1 - len(set([i[0] for i in thisTuple]))
        thisProb = np.product([i[1] for i in thisTuple])
        if thisCount == numReel:
            probDict[str(thisTuple[0][0]) * numReel] = thisProb
        probDict[thisCount] += thisProb
    return probDict

def probDictToSolidity(probDict, payoutKeys):
    payoutString = ""
    numPayout = len(payoutKeys)
    for thisKey in payoutKeys:
        thisProb = probDict[thisKey]
        thisString = "    if (maxMatch == %s) {\n      return betAmount * %s;\n    }\n" % (thisKey, str(int(1./(thisProb * numPayout))))
        payoutString += thisString
    return payoutString

def makeMachineToSolidity(machine):
    flatMachine = [item for sublist in machine for item in sublist]
    precision = max([10**(len(str(i[1]).split(".")[1])) for i in flatMachine])
    solidityString = "function makeMachine() internal {\n"
    for index, reel in enumerate(machine):
        reelString0 = "    reels[%s].probDenominator = %s;\n" % (str(index), str(precision))
        reelString1 = "    reels[%s].probs = %s;\n" % (str(index), str([int(i[1] * precision) for i in reel]))
        reelString2 = "    reels[%s].eventLabels = %s;\n" % (str(index), str([i[0] for i in reel]))
        
        reelString = reelString0 + reelString1 + reelString2
        solidityString += reelString
    return solidityString + "}"



In [3]:
numSymbolOnReel = [10,10,10,10,10]
numSymbolTotal = 20
precision = 1000

numReel = len(numSymbolOnReel)

machineOne = makeMachine(numSymbolOnReel, numSymbolTotal, precision)

Probabilities sum to 1? [1.0, 1.0, 1.0000000000000002, 1.0000000000000002, 1.0]


In [4]:
probDict = evaluateMachine(machineOne)
print(probDict)
print(sum([i[1] for i in probDict.items() if type(i[0]) == int]))

{1: 0.5881871851878981, 2: 0.3672459684087343, 3: 0.0437485591428482, 4: 0.0008178743994880002, 5: 4.1286102e-07, '1616161616': 3.58182e-07, '1212121212': 1.2175019999999998e-08, '99999': 4.2504e-08}
0.9999999999999887


In [5]:
payoutString = probDictToSolidity(probDict,[4,5])

In [6]:
def makeMachineToSolidity(machine):
    flatMachine = [item for sublist in machine for item in sublist]
    precision = max([10**(len(str(i[1]).split(".")[1])) for i in flatMachine])
    solidityString = "function makeMachine() internal {\n"
    for index, reel in enumerate(machine):
        reelString0 = "    reels[%s].probDenominator = %s;\n" % (str(index), str(precision))
        reelString1 = "    reels[%s].probs = %s;\n" % (str(index), str([int(i[1] * precision) for i in reel]))
        reelString2 = "    reels[%s].eventLabels = %s;\n" % (str(index), str([i[0] for i in reel]))
        
        reelString = reelString0 + reelString1 + reelString2
        solidityString += reelString
    return solidityString + "}"

In [7]:
makeMachineString = makeMachineToSolidity(machineOne)

In [8]:
solidityCode = '''
// ******************************
// DEFINE THE MACHINE

  // MACHINE SPECIFIC CONSTANTS AND VARIABLES
  uint8 constant numReel = %s;
  
  // DEFINE THE PROBABILITIES FOR THE REEL SYMBOLS
  %s
  
  // CALCULATE THE PAYOUT
  function paytable(uint[numReel] memory outcome, uint betAmount) internal returns (uint) {
    uint maxMatch = countMaxMatch(outcome);
%s}
  
// END MACHINE DEFINITION
// ***************************************
  
''' % (str(numReel), makeMachineString, payoutString)

In [9]:
print(solidityCode)


// ******************************
// DEFINE THE MACHINE

  // MACHINE SPECIFIC CONSTANTS AND VARIABLES
  uint8 constant numReel = 5;
  
  // DEFINE THE PROBABILITIES FOR THE REEL SYMBOLS
  function makeMachine() internal {
    reels[0].probDenominator = 1000;
    reels[0].probs = [335, 205, 133, 93, 86, 74, 39, 14, 11, 10];
    reels[0].eventLabels = [16, 8, 6, 11, 3, 17, 12, 15, 19, 9];
    reels[1].probDenominator = 1000;
    reels[1].probs = [494, 126, 108, 98, 92, 37, 31, 11, 2, 1];
    reels[1].eventLabels = [5, 6, 16, 1, 9, 7, 13, 12, 11, 10];
    reels[2].probDenominator = 1000;
    reels[2].probs = [331, 243, 119, 75, 60, 43, 40, 37, 35, 17];
    reels[2].eventLabels = [4, 3, 13, 1, 16, 12, 9, 18, 8, 19];
    reels[3].probDenominator = 1000;
    reels[3].probs = [234, 185, 156, 153, 137, 57, 40, 21, 11, 6];
    reels[3].eventLabels = [15, 7, 3, 19, 8, 2, 11, 9, 16, 12];
    reels[4].probDenominator = 1000;
    reels[4].probs = [365, 117, 110, 104, 95, 87, 55, 50, 15, 2];
    r